<a href="https://colab.research.google.com/github/albertovalverde/PersonaPlex-Robot/blob/main/PersonaPlex_Machetero_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PersonaPlex Full-Duplex Social Robot

```
    
```

**Cut through the noise.** Command Claude, Gemini, Realm, and shell with your voice.

**Requirements:** HuggingFace token (free), A100 GPU

## Step 1: Install Dependencies

In [ ]:
!nvidia-smi --query-gpu=name,memory.total --format=csv
print("\n✅ GPU ready!")

# System dependencies
!apt-get update -qq
!apt-get install -y -qq libopus-dev portaudio19-dev ffmpeg > /dev/null 2>&1
print("✅ System deps installed")

# PersonaPlex Machetero
!pip install -q git+https://github.com/chrisrijos/personaplex.git#subdirectory=moshi
!pip install -q torchaudio
print("✅ Machetero installed")

# Cloudflare tunnel (faster than localtunnel)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1
print("✅ Cloudflare tunnel installed")

# HuggingFace token from Colab secrets
# To set up: Click 🔑 (Secrets) in left sidebar → Add "HF_TOKEN"
import os
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ HF token loaded from Colab secrets")
except:
    from getpass import getpass
    HF_TOKEN = getpass("Paste your HuggingFace token: ")
    print("✅ HF token set manually")

os.environ["HF_TOKEN"] = HF_TOKEN
os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

## Step 2: Start Cloudflare Tunnel

Creates a public URL for your Mac to connect.

In [ ]:
import subprocess, re, time

# Kill any existing tunnels
!pkill -f cloudflared || true

# Start Cloudflare tunnel
proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8998"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

print("Starting tunnel...")
tunnel_url = None
for _ in range(30):
    line = proc.stdout.readline()
    if line:
        match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
        if match:
            tunnel_url = match.group(0)
            break
    time.sleep(1)

if tunnel_url:
    ws_url = tunnel_url.replace("https://", "wss://")
    print("\n" + "="*70)
    print("  🔥 MACHETERO TUNNEL READY 🔥")
    print("="*70)
    print(f"\n🌐 URL: {ws_url}/api/chat")
    print("\n📋 Run on your Mac:")
    print(f"\n   cd ~/Projects/personaplex")
    print(f"   python -m moshi.remote_client --server {ws_url}/api/chat")
    print("\n" + "="*70)
else:
    print("❌ Tunnel failed. Re-run this cell.")

## Step 5: Start Server

**Keep this cell running!** First run downloads model (~14GB, 3-5 min).

**Use headphones** to prevent audio feedback loop.

In [ ]:
# Start Machetero server (no SSL - Cloudflare handles it)
!python -m moshi.terminal_server \
    --host 0.0.0.0 \
    --port 8998 \
    --device cuda \
    --whisper-model base.en

---

## Voice Commands

| Say This | Routes To |
|----------|----------|
| "Claude, explain decorators" | Claude Code CLI |
| "Hey Claude, fix this bug" | Claude Code CLI |
| "Gemini, summarize this" | Gemini CLI |
| "Realm sync" | Realm CLI |
| "Run git status" | Shell |
| "Execute npm test" | Shell |
| "Help" | System |
| "Quit" | Exit |

---

## Troubleshooting

**Talking to itself:** Use headphones!

**Connection refused:** Wait for "Server running" in Step 5, then re-run Step 4 for new URL.

**Out of Memory:** Use A100 GPU (not T4).

**Session disconnects:** Keep browser tab active. Re-run Steps 4 + 5.